In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from typing import List, Tuple, Dict
from dataclasses import dataclass
import random
from torch.utils.data import Dataset, DataLoader

In [ ]:
@dataclass
class FacultyParams:
    """Parameters for each faculty"""
    name: str
    utility_vector: np.ndarray  # Hidden vector that determines student success
    capacity: int  # Number of spots available (can be infinite)

@dataclass
class SupplierParams:
    """Parameters for each preparation supplier"""
    name: str
    diff_vector: np.ndarray  # How this supplier modifies student features
    cost: float  # Cost of the service
    reputation: float  # Current reputation score

class UniversityEnvironment:
    def __init__(
        self,
        n_features: int = 5,  # Number of student features (e.g., math, english, etc.)
        n_faculties: int = 3,  # Number of different faculties
        n_suppliers: int = 4,  # Number of preparation suppliers
        noise_range: Tuple[float, float] = (-5, 5)  # Range for uniform noise
    ):
        self.n_features = n_features
        self.n_faculties = n_faculties
        self.n_suppliers = n_suppliers
        self.noise_range = noise_range
        
        # Initialize faculties with random utility vectors
        self.faculties = [
            FacultyParams(
                name=f"Faculty_{i}",
                utility_vector=np.random.normal(0, 1, n_features),
                capacity=np.inf  # As per description, infinite capacity
            )
            for i in range(n_faculties)
        ]
        
        # Initialize suppliers with random modification vectors
        self.suppliers = [
            SupplierParams(
                name=f"Supplier_{i}",
                diff_vector=np.random.normal(0, 0.2, n_features),  # Small modifications
            )
            for i in range(n_suppliers)
        ]
        
        self.past_applicants_df = None
        self.current_applicants_df = None

    def _calculate_final_grade(
        self,
        student_features: np.ndarray,
        faculty_idx: int
    ) -> float:
        """Calculate final grade for a student in a specific faculty"""
        faculty_vector = self.faculties[faculty_idx].utility_vector
        base_grade = np.dot(student_features, faculty_vector)
        noise = np.random.uniform(*self.noise_range)
        return base_grade + noise

    def generate_past_applicants(
        self,
        n_applicants: int = 1000
    ) -> pd.DataFrame:
        """Generate dataset of past applicants with their outcomes"""
        # Generate random feature vectors
        features = np.random.normal(0, 1, (n_applicants, self.n_features))
        
        # Calculate grades for all faculties
        grades = np.zeros((n_applicants, self.n_faculties))
        # Reshape faculty vectors into (n_faculties, n_features) matrix
        faculty_vectors = np.array([f.utility_vector for f in self.faculties])
        # Calculate base grades using matrix multiplication
        base_grades = np.dot(features, faculty_vectors.T)
        # Add uniform noise to all grades at once
        noise = np.random.uniform(*self.noise_range, size=(n_applicants, self.n_faculties))
        grades = base_grades + noise
        
        # Create DataFrame
        feature_cols = [f"feature_{i}" for i in range(self.n_features)]
        faculty_grade_cols = [f"faculty_{i}_grade" for i in range(self.n_faculties)]
        
        df = pd.DataFrame(features, columns=feature_cols)
        df_grades = pd.DataFrame(grades, columns=faculty_grade_cols)
        
        # Add best faculty (argmax of grades)
        df['assigned_faculty'] = np.argmax(grades, axis=1)
        df['final_grade'] = df_grades.max(axis=1)
        
        self.past_applicants_df = df
        return df

    def generate_current_applicants(
        self,
        n_applicants: int = 100
    ) -> pd.DataFrame:
        """Generate dataset of current applicants"""
        # Generate random feature vectors
        features = np.random.normal(0, 1, (n_applicants, self.n_features))
        
        # Create DataFrame
        feature_cols = [f"feature_{i}" for i in range(self.n_features)]
        df = pd.DataFrame(features, columns=feature_cols)
        
        # Add desired faculty (random)
        df['desired_faculty'] = np.random.randint(0, self.n_faculties, n_applicants)
        
        self.current_applicants_df = df
        return df

    def recommend(
        self,
        student_features: np.ndarray,
        recommended_faculty: int
    ) -> float:
        """Calculate final grade for a student given their features and recommended faculty"""
        return self._calculate_final_grade(student_features, recommended_faculty)

class UniversityMLP(nn.Module):
    """Simple MLP for university decisions"""
    def __init__(self, n_features: int, n_faculties: int):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(n_features, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, n_faculties)
        )
    
    def forward(self, x):
        return self.network(x)

class ApplicantMLP(nn.Module):
    """MLP for applicant decisions with softmax output"""
    def __init__(self, n_features: int, n_faculties: int):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(n_features, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, n_faculties),
            nn.Softmax(dim=1)
        )
    
    def forward(self, x):
        return self.network(x)

In [ ]:
# Example usage:
def run_example():
    # Create environment
    env = UniversityEnvironment()
    
    # Generate past applicants
    past_df = env.generate_past_applicants(1000)
    print("Past applicants shape:", past_df.shape)
    
    # Generate current applicants
    current_df = env.generate_current_applicants(100)
    print("Current applicants shape:", current_df.shape)
    
    # Example recommendation
    student_features = current_df.iloc[0][:[f"feature_{i}" for i in range(env.n_features)]].values
    recommended_faculty = 0
    final_grade = env.recommend(student_features, recommended_faculty)
    print(f"Example student final grade in faculty {recommended_faculty}: {final_grade}")